In [1]:
% matplotlib notebook
from my_utils import *
import numpy as np
import pandas
import torch, torchvision
import os

In [2]:
full_model = torch.load('./full_model.pt')

# define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
full_model.to(device);

In [3]:
filenames = os.listdir('./data/test/')

In [4]:
MEAN = [0.485, 0.456, 0.406] # expected by pretrained resnet18
STD = [0.229, 0.224, 0.225] # expected by pretrained resnet18

# define transformations without data augmentation
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(), # Expected by pretrained neural network
    torchvision.transforms.Normalize(MEAN, STD) # Expected by pretrained neural network
    ])

In [6]:
test_df = pandas.DataFrame({'Id':filenames})

In [7]:
test_data = TestDataset(test_df,'./data/test/',transform=transforms)

In [8]:
# load the trainind data to get a list of the categories
train_df = pandas.read_csv('./data/train.csv')
train_data = WhaleDataset(train_df,'./data/',transform=None)
categories = train_data.categories

In [9]:
test_dataloader = torch.utils.data.DataLoader(test_data,\
                                               batch_size=4,\
                                               num_workers=4,\
                                               shuffle=False,\
                                               sampler=None)

In [10]:
result = predict(full_model,test_dataloader,device,categories)

Elapsed time: 1885.4795


In [12]:
result.to_csv('./my_submission.csv',index=False)